<a href="https://colab.research.google.com/github/NaveenSanjaya/Right-Vote/blob/main/Social_Media_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Twitter Sentiment Analysis

In [ ]:
import tweepy
import re
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from scipy.special import softmax

# Twitter API credentials
API_KEY = 'MCu2BiFyXXXXXXX'
API_SECRET_KEY = 'Cl8FYiqoVl82KyXXXXXXXXXX'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAXXXXXXX'
ACCESS_TOKEN = '1709435XXXXXXXX'
ACCESS_TOKEN_SECRET = 'Lp50yuMNJjg5SDZXXXXXXXX'

# Set up Twitter API client
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Initialize sentiment analysis model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = TFAutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

# Labels for sentiment analysis
labels = ['Negative', 'Neutral', 'Positive']

def preprocess_tweet(tweet):
    """
    Preprocess the tweet by replacing mentions and URLs with placeholders.
    """
    tweet_words = []
    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)
    return " ".join(tweet_words)

def fetch_tweets(query, max_results=100):
    """
    Fetch recent tweets containing the specified query.
    """
    tweet_data = []
    try:
        tweets = client.search_recent_tweets(query=query, max_results=max_results, tweet_fields=['created_at', 'text'])
        if tweets.data:
            for tweet in tweets.data:
                tweet_data.append({
                    'created_at': tweet.created_at,
                    'text': tweet.text
                })
    except Exception as e:
        print(f"Error fetching tweets: {e}")
    return tweet_data

def analyze_sentiment(text):
    """
    Analyze the sentiment of the given text using the pre-trained model.
    """
    tweet_proc = preprocess_tweet(text)
    encoded_tweet = tokenizer(tweet_proc, return_tensors='tf')
    output = model(**encoded_tweet)
    scores = output[0][0].numpy()
    scores = softmax(scores)
    return dict(zip(labels, scores))

def plot_sentiment_distribution(sentiment_counts):
    """
    Plot a bar chart of sentiment distribution.
    """
    sentiments = list(sentiment_counts.keys())
    counts = list(sentiment_counts.values())
    plt.bar(sentiments, counts, color=['red', 'gray', 'green'])
    plt.xlabel('Sentiment')
    plt.ylabel('Count')
    plt.title('Sentiment Distribution')
    plt.show()

def main():
    # Example candidate names
    candidates = ["Sajith Premadasa", "Ranil Wickremesinghe", "Anura Kumara"]

    all_tweet_data = []

    for candidate in candidates:
        print(f"Fetching tweets for {candidate}...")
        tweets = fetch_tweets(candidate, max_results=100)
        for tweet in tweets:
            tweet['candidate'] = candidate
            all_tweet_data.append(tweet)

    # Convert tweet data to DataFrame
    df = pd.DataFrame(all_tweet_data)

    # Analyze sentiment for each tweet
    sentiment_counts = {label: 0 for label in labels}

    for text in df['text']:
        sentiment = analyze_sentiment(text)
        for label, score in sentiment.items():
            sentiment_counts[label] += score

    # Print sentiment distribution
    print("Sentiment Distribution:")
    for label, count in sentiment_counts.items():
        print(f"{label}: {count}")

    # Plot sentiment distribution
    plot_sentiment_distribution(sentiment_counts)

if __name__ == "__main__":
    main()


### Facebook Sentiment Analysis

In [ ]:
import requests
from transformers import pipeline

# Facebook API credentials
ACCESS_TOKEN = 'EAAjjdeAxAF1eT5ZBbibXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
PAGE_ID = 'XXXXXXXXXXXXX'

# Initialize sentiment analysis pipeline
sentiment_pipeline = pipeline('sentiment-analysis')

def fetch_facebook_posts(page_id, access_token, limit=10):
    """
    Fetch recent posts from a Facebook page.
    """
    url = f'https://graph.facebook.com/{page_id}/posts'
    params = {
        'access_token': access_token,
        'limit': limit,
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json().get('data', [])

def analyze_sentiment(text):
    """
    Analyze the sentiment of the given text using Hugging Face's sentiment analysis model.
    """
    return sentiment_pipeline(text)

def main():
    # Fetch recent posts
    posts = fetch_facebook_posts(PAGE_ID, ACCESS_TOKEN, limit=5)

    if not posts:
        print("No posts found.")
        return

    # Analyze sentiment for each post
    for post in posts:
        post_message = post.get('message', '')
        if post_message:
            sentiment = analyze_sentiment(post_message)
            print(f"Post: {post_message}")
            print(f"Sentiment: {sentiment[0]['label']} (Confidence: {sentiment[0]['score']:.2f})")
            print()

if __name__ == "__main__":
    main()
